# Pipeline for analysising expression data using SRIQ clustering

If you are having problems with the pipeline, refer to the github page

## Pre processing raw data

Importing the appropiate directories

In [ ]:
from DE import networkAnalysis as na

clusterDirectory need to point towards directory in which all the clusters can be found. 
It will then loop through all of the data files, gathering the sample ids from all cluster and create individual .csv files for analysis in R

In [ ]:
clusterDirectory = "data/expressionData/filtered(27k).txt"
obj = na()
obj.readCsv(clusterDirectory)


In [ ]:
obj.preProcess()

In [ ]:
#Set which bottom percentage you intend to filter away
bottom = 0.5
obj.filterVariantGenes(bottom = bottom)

In [ ]:
#Writes the pre processed and filtered, MUST be .txt and tab separated values
obj.filterDf.to_csv(f'data/expressionData/tcga_log2med.txt', sep = '\t', index = False)
obj.DEGDf.to_csv(f'data/expressionData/DGEA.txt', sep = '\t', index = False)

Configure test.properties and run VRLA using following command in the same directory as VRLA:

In [ ]:
studyName='LUAD_ens_q'
studyPath='/Users/jacobkarlstrom/projekt/SRIQ/notebook/data/expressionData/'
inFileName='tcga_log2med'
outPath='/Users/jacobkarlstrom/projekt/SRIQ/software/output/'
minBagSize=1200

obj.configResources(studyName=studyName, data = inFileName, )

Shell command running SRIQ

In [ ]:
!cd ../software/VRLA && java -jar VRLA.jar

## Expression analysis

In [1]:
from DE import networkAnalysis as na

csvpath = 'data/expressionData/fitered(27k).txt'
SRIQ_cluster_path = 'data/expressionData/publication_image_10000itr_1200var_10r/10000/QC_Spiral(false)/dist(0.63)/6'

obj = na()

obj.readSRIQ(csvpath, SRIQ_cluster_path)

[Errno 2] No such file or directory: 'data/expressionData/fitered(27k).txt'


## Visualing genes across clusters

Set scopes to your gene format according to mygene documentation

Takes single list of genes as argument

In [ ]:
obj.plotSingleGene(['KRAS', 'EGFR'], scopes = 'ensembl.gene')

Takes list of lists of genes as argument

In [ ]:
obj.plotMultipleGenes([['KRAS', 'EGFR']], scopes = 'ensembl.gene')

## Visualization of variation

In [ ]:
obj.SilhouttePlot(sizes=150)

In [ ]:
obj.Umap()

## SAM analysis

Before running this module, the entire SRIQ folder output has to be copied to the same folder in which the expression data table is located

In [ ]:
inFileName='DGEA'
studyName = 'publication_image'
#obj.configResources(studyName=studyName, data = inFileName, )

properties = '/Users/jacobkarlstrom/projekt/SRIQ/software/VRLA/resources/test.properties'
expressionData = '/Users/jacobkarlstrom/projekt/SRIQ/notebook/data/expressionData/DGEA-thresh.txt'
spiral = False
dist = 0.63
cmd = f'cd SAMDEG/dist && java -jar SAMDEG.jar {properties} {spiral} {dist} {int(obj.clusterNum)} 5 2 "{expressionData}"'

cmd

### Runs the SAM-analysis

In [ ]:
!$cmd

### Visualizing DEG

In [ ]:
resultsPath = '/Users/jacobkarlstrom/projekt/SRIQ/notebook/data/expressionData/publication_image_10000itr_1200var_10r/10000/QC_Spiral(false)/Results_log_0.63_6/publication_image_Data_in_6_ClusterOrder_ABS_Unique.txt'

obj.plotSamResults(resultsPath = resultsPath, q = 5, lfc = 2)


In [ ]:
obj.SilhouttePlot()

## Enrichment analysis

Converts the enrichment list to symbols if in ensembleIDs

In [ ]:
scopes = 'ensembl.gene'
obj.ensemble2gene(scopes)

Performs enrichment analysis from the differentially expressed genes.

In [ ]:
enrichDbs = ['GO_Biological_Process_2018', 'GO_Cellular_Component_2018', 'GO_Molecular_Function_2018']
obj.enrichR(dbs = enrichDbs)

Plots the enrichment results

In [ ]:
#up or down
u_d = 'up'

g = obj.plotEnrichmentResults(u_d=u_d, yticklabels =1)
